### Setup Models

In [31]:
# Import libraries

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

import pandas as pd

import os
import uuid
from dotenv import load_dotenv
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [4]:
# Start the LLM and embeddings
model = ChatOpenAI(
    model = "gpt-4o",
    openai_api_key = OPENAI_API_KEY
)

embedding = OpenAIEmbeddings(
    model = "text-embedding-3-large",
)

### Add Product Pricing Function tool

#### Load pricing file

In [9]:
# Load the laptop product pricing CSV into a Pandas Dataframe
product_pricing_df = pd.read_csv("..//data//Laptop pricing.csv")
product_pricing_df.head()

,Name,Price,ShippingDays
0,AlphaBook Pro,1499,2
1,GammaAir X,1399,7
2,SpectraBook S,2499,7
3,OmegaPro G17,2199,14
4,NanoEdge Flex,1699,2


#### Create the tool to get laptop price

In [10]:
@tool
def get_laptop_price(laptop_name:str) -> int:
    """
    This function returns the price of a laptop, given it's name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the price of the laptop.
    If there is no match found, it returns -1
    """
    # Filter Dataframe for matching names
    match_records_df = product_pricing_df[product_pricing_df["Name"].str.contains("^" + laptop_name, case = False)]

    if len(match_records_df) == 0:
        return -1
    else:
        return match_records_df["Price"].iloc[0]

In [11]:
print(get_laptop_price("omega"))
print(get_laptop_price("sky"))

2199
-1


C:\Users\akash\AppData\Local\Temp\ipykernel_920\4103377959.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(get_laptop_price("omega"))


### Add Product features retrieval tool

#### Load, chunk and index the contents of product feature PDF

In [19]:
loader = PyPDFLoader("..//data//Laptop product descriptions.pdf")
docs = loader.load()

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1024, chunk_overlap = 256)
splits = text_splitter.split_documents(docs)

#### Create a vector store with Chroma

In [18]:
prod_feature_store = Chroma.from_documents(
    documents = splits,
    embedding = embedding
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


#### Create the retriever

In [21]:
get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs = {"k":1}),
    name = "Get_Product_Features",
    description = """
    This store contains product features about laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

In [22]:
print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(metadata={'page': 0, 'source': '..//data//Laptop product descriptions.pdf'}, page_content='Fictional Laptop Descriptions\nAlphaBook Pro\nThe AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,\nand a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of\npower and portability.\nGammaAir X\nGammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB\nNVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a\nportable design.\nSpectraBook S\nDesigned for power users, SpectraBook S features an Intel Core i9 processor, 64GB RAM, and a\nmassive 2TB SSD. This workstation-class laptop is perfect for intensive tasks like video editing and\n3D rendering.\nOmegaPro G17\nOmegaPro G17 is a gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB\nSSD. Designed for gamers, it features a 17-inch display with a high refresh rate and powerful\ngraph

### Setup a simple Product QnA Chatbot

#### Create the system prompt

In [24]:
system_prompt = SystemMessage(
    """
    You are a professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses
    """
)

#### Create a list of tools available

In [25]:
tools = [get_laptop_price, get_product_features]

#### Create memory across questions in conversation

In [29]:
checkpointer = MemorySaver()

#### Create a product QnA Agent. This is actually a graph in LangGraph

In [30]:
product_QnA_Agent = create_react_agent(
    model = model, # llm
    tools = tools, # tools
    state_modifier = system_prompt, # System Prompt
    debug = False,
    checkpointer = checkpointer # For conversation memory
)

### Setup the chatbot

In [33]:
#Setup chatbot

#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_Agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_A2EzZD9au8kMTlhJdM4pDVeE)
 Call ID: call_A2EzZD9au8kMTlhJdM4pDVeE
  Args:
    query: GammaAir
  get_laptop_price (call_0pggK8r0eOcKEXnsxEwM1FyP)
 Call ID: call_0pggK8r0eOcKEXnsxEwM1FyP
  Args:
    laptop_name: GammaAir
================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

The GammaAir X laptop features an AMD Ryzen 7 processor, 32GB of DDR4 memory, and a 512GB NVMe SSD. It is designed with a thin and light form factor, making it an excellent choice for users who need high performance in a portable design.

The price for the GammaAir X is $1,399.


### Execute in chat format

In [35]:
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_Agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you with our laptops today?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! I can help you with that. Do you have any specific features or requirements in mind for your new laptop?
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here are some of the laptops we currently have available:

1. **AlphaBook Pro**: A sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM, and a fast 1TB SSD.

2. **GammaAir X**: Combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB NVMe SSD.

3. **SpectraBook S**: Features an Intel Core i9 processor, 64GB RAM, and a massive 2TB SSD, ideal for power users and intensive tasks.

4. **OmegaPro G17**: A gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB SSD, perfect for gamers.

5. **NanoEdge Flex**: (Details not provided, but it's part of our l

In [36]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_Agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")


USER 1: The SpectraBook S is designed for power users and is equipped with the following features:

- **Processor:** Intel Core i9
- **Memory:** 64GB RAM
- **Storage:** Massive 2TB SSD

This workstation-class laptop is perfect for intensive tasks like video editing and 3D rendering, making it an excellent choice for users requiring significant computational power.

USER 2: The GammaAir X is equipped with an AMD Ryzen 7 processor, 32GB of DDR4 memory, and a 512GB NVMe SSD. It features a thin and light design, making it ideal for users who require high performance in a portable package.

USER 1: The price of the SpectraBook S is $2,499.

USER 2: The price of the GammaAir X is $1,399.
